# Data Sourcing

## Imports

In [36]:
import os
import pandas as pd
import requests
from io import StringIO
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

## Web Scraping with Selenium

To conduct pattern analysis, the first thing to acquire is graphs that show patterns.

In order to do this, we will use Selenium to scrape the Atmatix website (www.atmatix.pl)

### Functions

### Run scraping

### Create a DataFrame from scraped data

### Save DF

## DataFrame 

From this cell on, all work is complete in the Notebook

### Load Dataframe

In [46]:
df = pd.read_csv('data/patterns.csv')

### Inspecting Dataframe

In [47]:
display(f'The Database has {df.shape[0]} rows and {df.shape[1]} columns.')
display('==================================================================================================================')
display(f'The Database has the following columns: {list(df.columns)}')
display('==================================================================================================================')
display(f"The database is missing {df['Company'].isna().sum()} Company names. That is {((df['Company'].isna().sum() / df.shape[0]) * 100):.2f}%")
display('==================================================================================================================')
display(f"The database is missing {df['Pattern'].isna().sum()} Patterns. That is {((df['Pattern'].isna().sum() / df.shape[0]) * 100):.2f}%")
display('==================================================================================================================')
display(f"The database is missing {df['Width'].isna().sum()} Widths. That is {((df['Width'].isna().sum() / df.shape[0]) * 100):.2f}%")
display('==================================================================================================================')
display(f"The database is missing {df['Breakout Date'].isna().sum()} Breakout dates. That is {((df['Breakout Date'].isna().sum() / df.shape[0]) * 100):.2f}%")
display('==================================================================================================================')
display(f"The database is missing {df['End Date'].isna().sum()} End dates. That is {((df['End Date'].isna().sum() / df.shape[0]) * 100):.2f}%")

'The Database has 3770 rows and 5 columns.'

'=================================================================================================================='

"The Database has the following columns: ['Company', 'Pattern', 'Width', 'End Date', 'Breakout Date']"

'=================================================================================================================='

'The database is missing 0 Company names. That is 0.00%'

...

'The database is missing 0 Widths. That is 0.00%'

'=================================================================================================================='

'The database is missing 2046 Breakout dates. That is 54.27%'

'=================================================================================================================='

'The database is missing 0 End dates. That is 0.00%'

### Adding a Ticker column

In [48]:
ticker_with_parentheses = df['Company'].str.extract(r'(?<=\()(\w+)(?=\))')
ticker_without_parentheses = df['Company'].str.extract(r'^(\w+)$')

df['Ticker'] = ticker_with_parentheses.fillna(ticker_without_parentheses)
df.head(3)

,Company,Pattern,Width,End Date,Breakout Date,Ticker
0,SANOK (SNK),H&S top,58,8/15/2023,NaN,SNK
1,GETIN (GTN),Vertical run up,13,8/3/2023,NaN,GTN
2,PEKAO (PEO),H&S top,47,8/30/2023,NaN,PEO


### Add API Start Date and API End Date column

In [49]:
df['End Date'] = pd.to_datetime(df['End Date'])

df['API End Date'] = df.apply(lambda row: row['End Date'] + pd.DateOffset(days=row['Width'] * 3), axis=1)
df['API Start Date'] = df.apply(lambda row: row['End Date'] - pd.DateOffset(days=row['Width'] * 4), axis=1)


df.head(3)

,Company,Pattern,Width,End Date,Breakout Date,Ticker,API End Date,API Start Date
0,SANOK (SNK),H&S top,58,2023-08-15,NaN,SNK,2024-02-05,2022-12-26
1,GETIN (GTN),Vertical run up,13,2023-08-03,NaN,GTN,2023-09-11,2023-06-12
2,PEKAO (PEO),H&S top,47,2023-08-30,NaN,PEO,2024-01-18,2023-02-23


In [50]:
df['API End Date'] = pd.to_datetime(df['API End Date'])
df['API Start Date'] = pd.to_datetime(df['API Start Date'])

## Using EOD HD API to retrieve stock information

## Inspecting New Data

### Add 'Have data' column

In [51]:
folder_name = 'data/patterns'

for index, row in df.iterrows():
    ticker = row.Ticker
    id_num = index
    pattern = '_'.join(word.lower() for word in row.Pattern.split())
    
    f_name = f'{index}_{ticker}_{pattern}.csv'
    f_path = os.path.join(folder_name, f_name)
    
    if os.path.exists(f_path):
        df.at[index, 'Have data'] = True
    else:
        df.at[index, 'Have data'] = False
        
df.head(3)

,Company,Pattern,Width,End Date,Breakout Date,Ticker,API End Date,API Start Date,Have data
0,SANOK (SNK),H&S top,58,2023-08-15,NaN,SNK,2024-02-05,2022-12-26,True
1,GETIN (GTN),Vertical run up,13,2023-08-03,NaN,GTN,2023-09-11,2023-06-12,True
2,PEKAO (PEO),H&S top,47,2023-08-30,NaN,PEO,2024-01-18,2023-02-23,True


### Missing data

In [52]:
missing_data['Have data'].value_counts()

Have data
False    778
Name: count, dtype: int64

In [60]:
missing_data = df[df['Have data'] == False]
display(f'There are {missing_data.shape[0]} rows of missing data')
display(f'There are {df.shape[0] - missing_data.shape[0]} completed DataFrames.')
display(missing_data.head())
display(missing_data['Ticker'].value_counts())

'There are 778 rows of missing data'

'There are 2992 completed DataFrames.'

,Company,Pattern,Width,End Date,Breakout Date,Ticker,API End Date,API Start Date,Have data
12,WIG-CHEMIA,Double bottom,60,2023-08-28,NaN,NaN,2024-02-24,2022-12-31,False
24,WIG-NRCHOM,Double top,11,2023-08-02,NaN,NaN,2023-09-04,2023-06-19,False
29,WIG-ENERG,H&S top,10,2023-08-30,NaN,NaN,2023-09-29,2023-07-21,False
34,WIG30,Flag,25,2023-08-11,NaN,WIG30,2023-10-25,2023-05-03,False
44,WIG-POLAND,Flag,25,2023-08-11,NaN,NaN,2023-10-25,2023-05-03,False


Ticker
WIGDIV        42
MWIG40        39
SWIG80        38
WIGTECH       37
WIG30         36
WIG           36
WIG20         35
FW20          30
WIG20SHORT    29
Name: count, dtype: int64

In [102]:
for index, row in df.iterrows():
    ticker = row.Ticker
    id_num = index
    pattern = '_'.join(word.lower() for word in row.Pattern.split())
    folder_name = f'data/patterns/{pattern}'
    
    f_name = f'{index}_{ticker}_{pattern}.csv'
    f_path = os.path.join(folder_name, f_name)
    
    if os.path.exists(f_path):
        df.at[index, 'Have data'] = True
    else:
        df.at[index, 'Have data'] = False
        
df.head(3)

,Company,Pattern,Width,End Date,Breakout Date,Ticker,API End Date,API Start Date,Have data
0,SANOK (SNK),H&S top,58,2023-08-15,NaN,SNK,2024-02-05,2022-12-26,True
1,GETIN (GTN),Vertical run up,13,2023-08-03,NaN,GTN,2023-09-11,2023-06-12,True
2,PEKAO (PEO),H&S top,47,2023-08-30,NaN,PEO,2024-01-18,2023-02-23,True


In [103]:
missing_data = df[df['Have data'] == False]
display(f'There are {missing_data.shape[0]} rows of missing data')
display(f'There are {df.shape[0] - missing_data.shape[0]} completed DataFrames.')
display(missing_data.head())
display(missing_data['Ticker'].value_counts())

'There are 631 rows of missing data'

'There are 3139 completed DataFrames.'

,Company,Pattern,Width,End Date,Breakout Date,Ticker,API End Date,API Start Date,Have data
12,WIG-CHEMIA,Double bottom,60,2023-08-28,NaN,NaN,2024-02-24,2022-12-31,False
24,WIG-NRCHOM,Double top,11,2023-08-02,NaN,NaN,2023-09-04,2023-06-19,False
29,WIG-ENERG,H&S top,10,2023-08-30,NaN,NaN,2023-09-29,2023-07-21,False
44,WIG-POLAND,Flag,25,2023-08-11,NaN,NaN,2023-10-25,2023-05-03,False
48,WIG-NRCHOM,Pennant,38,2023-08-22,NaN,NaN,2023-12-14,2023-03-23,False


Ticker
WIGDIV        42
MWIG40        39
WIG20         35
FW20          30
WIG20SHORT    29
Name: count, dtype: int64